In [5]:
# Импорт Pandas для работы с DataFrame
import pandas as pd

In [7]:
# Чтение CSV-файла в DataFrame
initial_df = pd.read_csv("Lab3.csv")

# Вывод первых 5 строк для проверки
print(initial_df.head())

               country  child_mort  exports  health  imports  income  \
0          Afghanistan        90.2     10.0    7.58     44.9    1610   
1              Albania        16.6     28.0    6.55     48.6    9930   
2              Algeria        27.3     38.4    4.17     31.4   12900   
3               Angola       119.0     62.3    2.85     42.9    5900   
4  Antigua and Barbuda        10.3     45.5    6.03     58.9   19100   

   inflation  life_expec  total_fer   gdpp  
0       9.44        56.2       5.82    553  
1       4.49        76.3       1.65   4090  
2      16.10        76.5       2.89   4460  
3      22.40        60.1       6.16   3530  
4       1.44        76.8       2.13  12200  


In [9]:
# удаление ненужного столбца
df = initial_df.drop(columns=['country'])

In [10]:
# Вывод количества пропусков по всем столбцам
print("Пропущенные значения по всем столбцам:")
print(df.isnull().sum())

Пропущенные значения по всем столбцам:
child_mort    0
exports       0
health        0
imports       0
income        0
inflation     0
life_expec    0
total_fer     0
gdpp          0
dtype: int64


In [11]:
# Обрабатываем признаки с инверсией (ниже = хуже → преобразуем в "выше = хуже")
df['life_expec_inv'] = df['life_expec'].max() - df['life_expec']
df['gdpp_inv'] = df['gdpp'].max() - df['gdpp']
df['income_inv'] = df['income'].max() - df['income']
df['health_inv'] = df['health'].max() - df['health']

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#Нормализуем данные
df_scaled = scaler.fit_transform(df)

In [14]:
from sklearn.cluster import KMeans

#Обучаем модель для кластеризации на основе k-средних
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df['cluster_kmeans'] = kmeans.fit_predict(df_scaled)

In [15]:
print(df)

     child_mort  exports  health  imports  income  inflation  life_expec  \
0          90.2     10.0    7.58     44.9    1610       9.44        56.2   
1          16.6     28.0    6.55     48.6    9930       4.49        76.3   
2          27.3     38.4    4.17     31.4   12900      16.10        76.5   
3         119.0     62.3    2.85     42.9    5900      22.40        60.1   
4          10.3     45.5    6.03     58.9   19100       1.44        76.8   
..          ...      ...     ...      ...     ...        ...         ...   
162        29.2     46.6    5.25     52.7    2950       2.62        63.0   
163        17.1     28.5    4.91     17.6   16500      45.90        75.4   
164        23.3     72.0    6.84     80.2    4490      12.10        73.1   
165        56.3     30.0    5.18     34.4    4480      23.60        67.5   
166        83.1     37.0    5.89     30.9    3280      14.00        52.0   

     total_fer   gdpp  life_expec_inv  gdpp_inv  income_inv  health_inv  \
0         5.

In [16]:
# Средние по кластерам
summary = df.groupby('cluster_kmeans')[['child_mort', 'gdpp', 'life_expec', 'income', 'total_fer']].mean()
print(summary)

                child_mort          gdpp  life_expec        income  total_fer
cluster_kmeans                                                               
0                90.557143   1928.612245   59.365306   3885.979592   4.960816
1                 4.903125  45331.250000   80.484375  47475.000000   1.774062
2                20.894186   7208.279070   73.237209  13413.372093   2.237907


In [22]:
# Найдем страну, ближайшая к центроиду кластера

from sklearn.metrics.pairwise import euclidean_distances

centroids = kmeans.cluster_centers_
distances = euclidean_distances(df_scaled, centroids)

representative_countries = []

for cluster_id in range(kmeans.n_clusters):
    mask = df['cluster_kmeans'] == cluster_id
    if mask.sum() == 0:
        representative_countries.append("Нет стран")
        continue

    cluster_distances = distances[mask, cluster_id]
    closest_idx_in_cluster = cluster_distances.argmin()

    # Получаем ИСТИННЫЙ ИНДЕКС строки в исходном df (и initial_df)
    original_index = df[mask].index[closest_idx_in_cluster]

    # Берём название страны из initial_df по этому индексу
    country_name = initial_df.loc[original_index, 'country']
    representative_countries.append(country_name)

print("Эталонные страны по кластерам:", representative_countries)


Эталонные страны по кластерам: ['Tanzania', 'Iceland', 'Paraguay']


In [23]:
# Выберираем ключевые социально-экономические признаки
key_features = ['child_mort', 'gdpp', 'life_expec', 'income', 'health', 'total_fer']

cluster_summary = df.groupby('cluster_kmeans')[key_features].mean()
print(cluster_summary)

                child_mort          gdpp  life_expec        income    health  \
cluster_kmeans                                                                 
0                90.557143   1928.612245   59.365306   3885.979592  6.592041   
1                 4.903125  45331.250000   80.484375  47475.000000  9.015937   
2                20.894186   7208.279070   73.237209  13413.372093  6.124419   

                total_fer  
cluster_kmeans             
0                4.960816  
1                1.774062  
2                2.237907  


кластер с самыми высокими значениями по всем колонкам — это и есть группа стран, которым нужна помощь в первую очередь (так как была инверсия).